In [10]:
from missing_bids import auction_data, analytics, environments
hist_plot = auction_data.hist_plot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('white')
%matplotlib inline

In [11]:
tsuchiura_data = auction_data.AuctionData('../tests/reference_data/tsuchiura_data.csv')

tsuchiura_before_min_price = auction_data.AuctionData(
    tsuchiura_data.df_bids.loc[tsuchiura_data.data.minprice.isnull()])

In [12]:
filter_ties = auction_data.FilterTies(tolerance=.0001)
filtered_data = filter_ties(tsuchiura_before_min_price)

In [13]:
deviations = [-.02, .0, .0015] # [.0, .0008]
demands = [filtered_data.get_counterfactual_demand(rho) for rho in deviations]
print(demands)

[0.657617728531856, 0.21772853185595567, 0.2149584487534626]


In [14]:
constraints = [environments.MarkupConstraint(max_markup=.5, min_markup=.01), 
               environments.InformationConstraint(k=.5, sample_demands=demands)]

In [19]:
# result with seed 0

min_collusion_solver = analytics.MinCollusionIterativeSolver(
    data=tsuchiura_before_min_price, 
    deviations=deviations, 
    metric=analytics.IsNonCompetitive, 
    plausibility_constraints=constraints, 
    num_points=1000.0, 
    seed=0, 
    project=True, 
    filter_ties=None,
    number_iterations=500,
    confidence_level=.95,
    moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
    moment_weights=np.array([0, 0, 1])
)

print('solver demands: {}'.format(min_collusion_solver.demands))
print('rough tolerance T^2: {}'.format(4. / tsuchiura_before_min_price.df_bids.shape[0]))
print('bootstrapped tolerance T^2: {}'.format(min_collusion_solver.tolerance))

result = min_collusion_solver.result
print('minimum share of collusive auctions: {}'.format(result.solution))
result.argmin.head()

solver demands: [0.65899123 0.21792763 0.2127193 ]
rough tolerance T^2: 0.0021929824561403508
bootstrapped tolerance T^2: 9.092340912588504e-06
minimum share of collusive auctions: 0.43313431206954106


,-0.02,0.0,0.0015,cost,metric,prob
0,0.696554,0.171825,0.159112,0.980837,0.0,5.668657e-01
1,0.747599,0.154728,0.152381,0.945621,1.0,4.331343e-01
2,0.586474,0.180097,0.177709,0.936770,1.0,1.979669e-10
3,0.637473,0.197980,0.195434,0.711506,1.0,1.589892e-11
4,0.692348,0.169282,0.166708,0.983969,1.0,1.448258e-11


In [20]:
# result with seed 1

min_collusion_solver = analytics.MinCollusionIterativeSolver(
    data=tsuchiura_before_min_price, 
    deviations=deviations, 
    metric=analytics.IsNonCompetitive, 
    plausibility_constraints=constraints, 
    num_points=1000.0, 
    seed=1, 
    project=True, 
    filter_ties=None,
    number_iterations=500,
    confidence_level=.95,
    moment_matrix=auction_data.moment_matrix(deviations, 'slope'),
    moment_weights=np.array([0, 0, 1])
)

print('solver demands: {}'.format(min_collusion_solver.demands))
print('rough tolerance T^2: {}'.format(4. / tsuchiura_before_min_price.df_bids.shape[0]))
print('bootstrapped tolerance T^2: {}'.format(min_collusion_solver.tolerance))

result = min_collusion_solver.result
print('minimum share of collusive auctions: {}'.format(result.solution))
result.argmin.head()

solver demands: [0.65899123 0.21792763 0.2127193 ]
rough tolerance T^2: 0.0021929824561403508
bootstrapped tolerance T^2: 9.092340912588504e-06
minimum share of collusive auctions: 0.4354237511996902


,-0.02,0.0,0.0015,cost,metric,prob
0,0.592016,0.172746,0.160043,0.976881,0.0,5.645762e-01
1,0.618059,0.148133,0.145716,0.705606,1.0,4.354237e-01
2,0.730595,0.155162,0.152740,0.952311,1.0,2.766409e-08
3,0.733666,0.184400,0.181844,0.977717,1.0,2.109497e-10
4,0.693012,0.170511,0.167905,0.985866,1.0,2.054493e-10
